## 🧪 Lab 8: 1D CNN for Hour-Ahead Short-Term Load Forecasting (STLF)

## 🎯 Objectives

The primary goals of this section are:

1. **Design and implement a 1D Convolutional Neural Network (1D-CNN)** tailored for time-series data related to electrical load forecasting.

2. **Perform hour-ahead Short-Term Load Forecasting (STLF)** using historical load and auxiliary features to predict the next hour's demand.

3. **Extract temporal patterns** from sequential data using convolutional layers, improving model accuracy and generalization.

4. **Preprocess and structure time-series data** into suitable input-output sequences for CNN training.

5. **Evaluate model performance** using metrics such as MAE (Mean Absolute Error), RMSE (Root Mean Squared Error), and MAPE (Mean Absolute Percentage Error).

6. **Compare results** with traditional models (e.g., MLP or statistical baselines) to highlight the effectiveness of CNN in capturing local temporal features.

7. **Apply regularization and tuning techniques** (e.g., dropout, learning rate adjustment, batch normalization) to enhance model robustness and prevent overfitting.



### 📁 Changing Working Directory to Lab 8 Folder



In [1]:
import os
os.chdir(r'C:\Users\PMLS\labreports\lab8')

### 🧠 Importing Libraries and Custom Modules for Time-Series Modeling

This section includes all essential libraries, utility functions, custom callbacks, and deep learning components required to build, train, monitor, and evaluate time-series machine learning models. It covers:

- 📦 Data preprocessing and splitting
- 🏗️ Model architectures (LSTM, CNN, Bidirectional, etc.)
- 🧩 Custom callbacks and learning rate schedulers
- 🧮 Evaluation metrics (MAE, RMSE, R²)
- 🧠 Keras & TensorFlow-based deep learning utilities


In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

### ⚙️ Model Initialization and Configuration Parameters


In [3]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

### 🧠 CNN Model Architecture for Time-Series Regression


In [4]:
def CNN():
    input_data = Input(shape=(time_steps, num_features))
    x1 = Conv1D(16, 2, activation="relu")(input_data)
    x2 = Conv1D(16, 2, activation="relu")(x1)
    flatten = Flatten()(x2)
    output_data = Dense(1)(flatten)
    model = Model(input_data, output_data)
    return model

In [5]:
model1 = CNN()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 23, 16)              │             688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 22, 16)              │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 352)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             353 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,569 (6.13 KB)

 Trainable params: 1,569 (6.13 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
tensorflow.keras.utils.plot_model(model1 )

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


### 💾 File Paths for Checkpoints and Training History

Defines paths to save:

- Model checkpoints after each epoch
- Training history plot (`history.png`)
- Training history in JSON format (`history.json`)


In [11]:
checkpoints = r'C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\PMLS\labreports\lab8\CHECKPOINT'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

### 📉 Callbacks for Checkpointing and Training Monitoring


In [12]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

### 🚀 Model Compilation or Checkpoint Loading


In [13]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =CNN()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


### 📥 Loading Dataset and Scaler for Training, Validation, and Testing


In [16]:
import os
path_dataset =r'C:\Users\PMLS\labreports\lab8'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

### 🔧 Time-Series Input Configuration


In [17]:
time_steps=24
num_features=21

### 🧪 Preparing Input Data Using Univariate Multi-Step Function


In [18]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.3880305290222168 sec


### 🏋️ Model Training with Validation and Callbacks


In [19]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/60
2643/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0681 - mae: 0.0681 - mape: 4118.6855
Epoch 1: val_loss improved from inf to 0.02056, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.0680 - mae: 0.0680 - mape: 4113.2026 - val_loss: 0.0206 - val_mae: 0.0206 - val_mape: 9.5549
Epoch 2/60
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0193 - mae: 0.0193 - mape: 76.3419
Epoch 2: val_loss improved from 0.02056 to 0.02047, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E1-cp-0002-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0193 - mae: 0.0193 - mape: 76.6689 - val_loss: 0.0205 - val_mae: 0.0205 - val_mape: 9.2032
Epoch 3/60
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0149 - mae: 0.0149 - mape: 201.8743
Epoch 3: val_loss improved from 0.02047 to 0.01233, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0149 - mae: 0.0149 - mape: 201.9566 - val_loss: 0.0123 - val_mae: 0.0123 - val_mape: 6.1194
Epoch 4/60
2638/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0130 - mae: 0.0130 - mape: 208.5480
Epoch 4: val_loss did not improve from 0.01233
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0130 - mae: 0.0130 - mape: 209.7261 - val_loss: 0.0139 - val_mae: 0.0139 - val_mape: 7.8666
Epoch 5/60
2630/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0119 - mae: 0.0119 - mape: 693.5286
Epoch 5: val_loss improved from 0.01233 to 0.01192, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E1-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0119 - mae: 0.0119 - mape: 692.6422 - val_loss: 0.0119 - val_mae: 0.0119 - val_mape: 6.6328
Epoch 6/60
2643/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0107 - mae: 0.0107 - mape: 225.3252
Epoch 6: val_loss improved from 0.01192 to 0.01007, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E1-cp-0006-loss0.01.h5


...

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0088 - mae: 0.0088 - mape: 766.0668 - val_loss: 0.0080 - val_mae: 0.0080 - val_mape: 5.3320
Epoch 21/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 148.4272
Epoch 21: val_loss did not improve from 0.00802
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 148.6568 - val_loss: 0.0125 - val_mae: 0.0125 - val_mape: 7.1933
Epoch 22/60
2623/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0088 - mae: 0.0088 - mape: 742.5221
Epoch 22: val_loss improved from 0.00802 to 0.00785, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E1-cp-0022-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0088 - mae: 0.0088 - mape: 742.7806 - val_loss: 0.0078 - val_mae: 0.0078 - val_mape: 5.3041
Epoch 23/60
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 79.2037
Epoch 23: val_loss did not improve from 0.00785
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 81.1357 - val_loss: 0.0081 - val_mae: 0.0081 - val_mape: 5.2520
Epoch 24/60
2640/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0087 - mae: 0.0087 - mape: 306.9302
Epoch 24: val_loss did not improve from 0.00785
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0087 - mae: 0.0087 - mape: 308.8611 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 5.9659
Epoch 25/60
2623/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0083 - mae: 0.0083 - mape: 539.7932
Epoch 25: val_loss did not improve from 0.00785
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0083 - mae: 0.0083 - mape: 541.2460 - val_loss: 0.0084 - val_mae: 0

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 383.7497 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 5.2162
Epoch 33/60
2632/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 153.9827
Epoch 33: val_loss did not improve from 0.00772
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 159.3192 - val_loss: 0.0085 - val_mae: 0.0085 - val_mape: 5.7045
Epoch 34/60
2635/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0085 - mae: 0.0085 - mape: 794.0331
Epoch 34: val_loss did not improve from 0.00772
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0085 - mae: 0.0085 - mape: 793.2089 - val_loss: 0.0082 - val_mae: 0.0082 - val_mape: 5.4264
Epoch 35/60
2643/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0082 - mae: 0.0082 - mape: 554.7556
Epoch 35: val_loss did not improve from 0.00772
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0082 - mae: 0.0082 - mape: 555.6064 - val_loss: 0.0082 - val_mae:

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0083 - mae: 0.0083 - mape: 4101.8740 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 5.0878
Epoch 37/60
2631/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0080 - mae: 0.0080 - mape: 159.9030
Epoch 37: val_loss did not improve from 0.00727
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0080 - mae: 0.0080 - mape: 164.4442 - val_loss: 0.0094 - val_mae: 0.0094 - val_mape: 5.8143
Epoch 38/60
2627/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0081 - mae: 0.0081 - mape: 1116.0905
Epoch 38: val_loss did not improve from 0.00727
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0081 - mae: 0.0081 - mape: 1113.2776 - val_loss: 0.0075 - val_mae: 0.0075 - val_mape: 4.8599
Epoch 39/60
2628/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0080 - mae: 0.0080 - mape: 354.3700
Epoch 39: val_loss did not improve from 0.00727
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0080 - mae: 0.0080 - mape: 358.9678 - val_loss: 0.0082 - val_m

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0079 - mae: 0.0079 - mape: 671.9561 - val_loss: 0.0072 - val_mae: 0.0072 - val_mape: 4.8588
Epoch 42/60
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0080 - mae: 0.0080 - mape: 266.0005
Epoch 42: val_loss did not improve from 0.00723
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0080 - mae: 0.0080 - mape: 266.6642 - val_loss: 0.0075 - val_mae: 0.0075 - val_mape: 5.0346
Epoch 43/60
2644/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0079 - mae: 0.0079 - mape: 609.4049
Epoch 43: val_loss did not improve from 0.00723
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0079 - mae: 0.0079 - mape: 609.8514 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 5.2737
Epoch 44/60
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0079 - mae: 0.0079 - mape: 107.2542
Epoch 44: val_loss did not improve from 0.00723
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0079 - mae: 0.0079 - mape: 108.8026 - val_loss: 0.0080 - val_mae:

### 📊 Model Evaluation on Test Data


In [21]:
from keras.models import load_model

model = load_model(r'C:\Users\PMLS\labreports\lab8\CHECKPOINT\E1-cp-0041-loss0.01.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Mean Absolute Error (MAE): 114.83
Median Absolute Error (MedAE): 90.89
Mean Squared Error (MSE): 23110.45
Root Mean Squared Error (RMSE): 152.02
Mean Absolute Percentage Error (MAPE): 0.8 %
Median Absolute Percentage Error (MDAPE): 0.63 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


### 📁 Checkpoint Configuration and Model Initialization


In [44]:
# your checkpoint path and model path
checkpoints = r'C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model_path = r'C:\Users\PMLS\labreports\lab8\CHECKPOINT\E1-cp-0041-loss0.01.h5'
start_epoch = 58
model = None  # Initialize model

### 🔄 Model Compilation or Loading with PC Architecture and Callbacks


In [47]:
import PC
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


C:\Users\PMLS\anaconda3\envs\myenv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [48]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0541 - mae: 0.0421 - mape: 69.5731 
Epoch 1: val_loss improved from inf to 0.02579, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0001-loss0.03.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0541 - mae: 0.0421 - mape: 69.9045 - val_loss: 0.0258 - val_mae: 0.0210 - val_mape: 9.4100
Epoch 2/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0189 - mae: 0.0147 - mape: 298.1694
Epoch 2: val_loss improved from 0.02579 to 0.01367, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0189 - mae: 0.0147 - mape: 298.1977 - val_loss: 0.0137 - val_mae: 0.0105 - val_mape: 4.3960
Epoch 3/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0143 - mae: 0.0113 - mape: 10.5417
Epoch 3: val_loss improved from 0.01367 to 0.01137, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0143 - mae: 0.0113 - mape: 10.5599 - val_loss: 0.0114 - val_mae: 0.0088 - val_mape: 3.9979
Epoch 4/10
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0126 - mae: 0.0102 - mape: 142.5979
Epoch 4: val_loss did not improve from 0.01137
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0126 - mae: 0.0102 - mape: 142.7149 - val_loss: 0.0117 - val_mae: 0.0096 - val_mape: 4.5023
Epoch 5/10
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0114 - mae: 0.0094 - mape: 10.1554
Epoch 5: val_loss improved from 0.01137 to 0.01058, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0005-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0114 - mae: 0.0094 - mape: 10.1728 - val_loss: 0.0106 - val_mae: 0.0089 - val_mape: 3.9968
Epoch 6/10
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0109 - mae: 0.0092 - mape: 472.2401  
Epoch 6: val_loss did not improve from 0.01058
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0109 - mae: 0.0092 - mape: 471.1767 - val_loss: 0.0107 - val_mae: 0.0092 - val_mape: 4.0340
Epoch 7/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0104 - mae: 0.0090 - mape: 19.0051
Epoch 7: val_loss improved from 0.01058 to 0.01045, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0007-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0104 - mae: 0.0090 - mape: 19.0051 - val_loss: 0.0104 - val_mae: 0.0091 - val_mape: 4.1923
Epoch 8/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0101 - mae: 0.0088 - mape: 52.2630
Epoch 8: val_loss improved from 0.01045 to 0.00894, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0008-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0101 - mae: 0.0088 - mape: 52.2814 - val_loss: 0.0089 - val_mae: 0.0077 - val_mape: 3.2859
Epoch 9/10
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0096 - mae: 0.0084 - mape: 4.0877 
Epoch 9: val_loss improved from 0.00894 to 0.00850, saving model to C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0009-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.0096 - mae: 0.0084 - mape: 4.1418 - val_loss: 0.0085 - val_mae: 0.0074 - val_mape: 3.2079
Epoch 10/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0096 - mae: 0.0085 - mape: 174.2573 
Epoch 10: val_loss did not improve from 0.00850
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 0.0096 - mae: 0.0085 - mape: 174.2093 - val_loss: 0.0089 - val_mae: 0.0079 - val_mape: 3.2760


### 📊 Model Evaluation on Test Data


In [50]:

model = load_model(r'C:\Users\PMLS\labreports\lab8\CHECKPOINT\\E2-cp-0008-loss0.01.h5',compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Mean Absolute Error (MAE): 126.87
Median Absolute Error (MedAE): 97.6
Mean Squared Error (MSE): 28573.09
Root Mean Squared Error (RMSE): 169.04
Mean Absolute Percentage Error (MAPE): 0.87 %
Median Absolute Percentage Error (MDAPE): 0.67 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
